In [16]:
import dash
import dash_bootstrap_components as dbc
from dash import html
from dash import dcc, dash_table
import plotly.express as px
from dash.dependencies import Input, Output
import pandas as pd
from lib import utils, db, layout
import plotly.graph_objects as go
from decimal import Decimal

eng = db.get_engine()

sql = """select distinct symbol from CANDLE_DATA;"""
df = pd.read_sql_query(sql, eng)
list_symbols = df["symbol"].to_list()

sql = """select symbol, [Datetime], [Open], [High], [Low], [Close], interval from CANDLE_DATA;"""
df_candle_data = pd.read_sql_query(sql, eng)

sql = """select symbol, [Datetime], Open__change, interval from CANDLE_CHANGE;"""
df_candle_change = pd.read_sql_query(sql, eng)

sql = """select symbol, [Datetime], correlation, interval, days_back_planned from CALCULATED_CORRELATION;"""
df_correlation = pd.read_sql_query(sql, eng)

sql = """select symbol, Datetime, value, VAR_TYPE, VAR_METHOD, interval, days_back_planned from FINANCE_VAR;"""
df_var = pd.read_sql_query(sql, eng)

In [2]:
# overview

import numpy as np

list_intervals  = ["1d", "1m"]
if 1 == 1:
    df = df_candle_change
    table_list_symbols = []
    table_list_entries = []
    table_list_min_dates = []
    table_list_max_dates = []
    table_list_mean = []
    table_list_median = []
    table_list_std = []
    for s in list_symbols:
        for i in list_intervals:
            table_list_symbols.append(s + " (" + i +")")
            dummy = df[(df["symbol"] == s) & (df["interval"] == i)]["Datetime"]
            open__change = df[(df["symbol"] == s) & (df["interval"] == i)]["Open__change"]
            if len(dummy) == 0:
                min_sym = "N/A"
                max_sym = "N/A"
                dummy_mean = "N/A"
                dummy_median = "N/A"
                dummy_std = "N/A"
            else:   
                min_sym = min(dummy)
                max_sym = max(dummy)
                dummy_mean = '%.2E' % Decimal(np.mean(open__change))
                dummy_median = '%.2E' % Decimal(np.median(open__change))
                dummy_std = '%.2E' % Decimal(np.std(open__change))
            
            table_list_entries.append(len(dummy))
            table_list_min_dates.append(min_sym)
            table_list_max_dates.append(max_sym)
            table_list_mean.append(dummy_mean)
            table_list_median.append(dummy_median)
            table_list_std.append(dummy_std)

    df = pd.DataFrame(
        {
            "symbol": table_list_symbols,
            "# Entries": table_list_entries,
            "Min-Date": table_list_min_dates,
            "Max-Date": table_list_max_dates,
            "Mean": table_list_mean,
            "Median": table_list_median,
            "STD": table_list_std,
        }
    )

    table = dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True)

In [32]:
sql = """select distinct interval, DAYS_BACK_PLANNED from CALCULATED_CORRELATION;"""
df_correlation_intervals_days = pd.read_sql_query(sql, eng)

In [39]:
print(df_correlation_intervals_days.iloc[0]["interval"], df_correlation_intervals_days.iloc[0]["DAYS_BACK_PLANNED"])
val_interval = df_correlation_intervals_days.iloc[0]["interval"]
print(val_interval)
list(set(df_correlation_intervals_days[df_correlation_intervals_days["interval"] == val_interval]["DAYS_BACK_PLANNED"]))

1m 2
1m


[1, 2, 5]

In [41]:
# correlation multi

import itertools

def get_scatters(df, interval, list_symbols, x_name, y_name, method):
    list_scatters = {}
    df = df[(df["symbol"].isin(list_symbols)) & (df["interval"] == interval)]
    print(df)
    if method == "change":
        for s in list_symbols:
            df_dummy = df[(df["symbol"] == s)]
            list_scatters[s] = go.Scatter(x=df_dummy[x_name], y=df_dummy[y_name],  mode='lines+markers', yaxis='y1')

    elif method == "correlation":
        for s in list_symbols:
            df_dummy = df[(df["symbol"] == "____".join(s))]
            if len(df_dummy) == 0:
                df_dummy = df[(df["symbol"] == "____".join(s[::-1]))]
            list_scatters[s] = go.Scatter(x=df_dummy[x_name], y=df_dummy[y_name],  mode='lines+markers', yaxis='y1')

    return list_scatters

def get_change_scatters(df_in, interval, list_symbols):
    list_scatters = {}
    for s in list_symbols:
        df = df_in[(df_in["symbol"] == s) & (df_in["interval"] == interval)]
        list_scatters[s] = go.Scatter(x=df["Datetime"], y=df["Open__change"], name = s, mode='lines+markers', yaxis='y2')
    return list_scatters

def get_correlation_scatters(df_in, interval, days_back, list_symbols):
    list_scatters = {}
    for s in list_symbols:
        n = "____".join(s)
        df = df_in[(df_in["symbol"] == n) & (df_in["interval"] == interval) & (df_in["days_back_planned"] == int(days_back))]
        if len(df) == 0:
            n = "____".join(s)
            df = df_in[(df_in["symbol"] == n) & (df_in["interval"] == interval) & (df_in["days_back_planned"] == int(days_back))]
        list_scatters[n] = go.Scatter(x=df["Datetime"], y=df["correlation"], name = n, mode='lines+markers', yaxis='y1')
    return list_scatters

def get_pair_combinations(stuff):
    l = []
    for L in range(len(stuff) + 1):
        for subset in itertools.combinations(stuff, L):
            if len(subset) == 2:
                l.append(subset)
    return l

# main
interval = "1d"
days_back = "30"
list_symbols = ["AAPL", "EURUSD=X", "GME", "BBBY"]

# correlation
symbol_pairs =  get_pair_combinations(list_symbols)
dict_correlation_scatters = get_correlation_scatters(df_correlation.copy(), interval, days_back, symbol_pairs)
list_correlation_scatters = list(dict_correlation_scatters.values())
layout_correlation = go.Layout(title='Correlation', yaxis=dict(title='Crude and Model'), yaxis2=dict(title='Moddel Difference', overlaying='y', side='right'))
fig_correlation = go.Figure(data=list_correlation_scatters, layout=layout_correlation)
fig_correlation.update_layout(grid_yside="right")
fig_correlation.update_layout(yaxis_range=[-4,4])
fig_correlation.show()

# change
dict_change_scatters = get_change_scatters(df_candle_change.copy(), interval, list_symbols)

# pairwise change + correlation
fig_change = {}
for k in dict_correlation_scatters:
    s = k.split("____")
    scatter_change_0 = dict_change_scatters[s[0]]
    scatter_change_1 = dict_change_scatters[s[1]]
    scatter_correlation = dict_correlation_scatters[k]
    layout_change = go.Layout(title='Correlation', yaxis=dict(title='Crude and Model'), yaxis2=dict(title='Moddel Difference', overlaying='y', side='right'))
    fig_change[k] = go.Figure(data=[scatter_change_0, scatter_change_1, scatter_correlation], layout=layout_change)
    fig_change[k].update_layout(yaxis_range=[-1,1])
    #fig_change[k].update_layout(yaxis2_range=[-1,1])
    fig_change[k].show()
    break


In [17]:
list_symbols = ["AAPL", "EURUSD=X"]
df_correlation[df_correlation["symbol"] == "____".join(list_symbols)]

,symbol,Datetime,correlation,interval,days_back_planned
14,AAPL____EURUSD=X,2015-11-19,1.000000,1d,30
15,AAPL____EURUSD=X,2015-11-19,1.000000,1d,90
16,AAPL____EURUSD=X,2015-11-19,1.000000,1d,100
73,AAPL____EURUSD=X,2015-11-29,0.385101,1d,30
74,AAPL____EURUSD=X,2015-11-29,0.385101,1d,90
...,...,...,...,...,...
16146,AAPL____EURUSD=X,2023-01-31,0.222223,1d,90
16147,AAPL____EURUSD=X,2023-01-31,0.064495,1d,100
16184,AAPL____EURUSD=X,2023-02-10,0.193698,1d,90
16185,AAPL____EURUSD=X,2023-02-10,0.222223,1d,100


In [59]:
# correlation pair

# main
interval = "1d"
days_back = 30
list_symbols = ["AAPL", "EURUSD=X"]

# correlation
symbol_pairs =  get_pair_combinations(list_symbols)
dict_correlation_scatters = get_correlation_scatters(df_correlation.copy(), interval, days_back, symbol_pairs)
list_correlation_scatters = list(dict_correlation_scatters.values())
layout_correlation = go.Layout(title='Correlation', yaxis=dict(title='Crude and Model'), yaxis2=dict(title='Moddel Difference', overlaying='y', side='right'))
fig_correlation = go.Figure(data=list_correlation_scatters, layout=layout_correlation)
fig_correlation.update_layout(grid_yside="right")
fig_correlation.update_layout(yaxis_range=[-4,4])
fig_correlation.show()

# correlation - x - y
change_0 = df_candle_change[df_candle_change["symbol"] == list_symbols[0]][["Datetime", "Open__change"]]
change_0.index = change_0["Datetime"]
change_1 = df_candle_change[df_candle_change["symbol"] == list_symbols[1]][["Datetime", "Open__change"]]
change_1.index = change_1["Datetime"]



df_change_0_1 = change_0.join(change_1, rsuffix='_right')
df_change_0_1 = df_change_0_1[(~df_change_0_1["Open__change"].isnull()) & (~df_change_0_1["Open__change_right"].isnull())]

# restrict times
df_change_0_1 = df_change_0_1.loc['2019-11-01':'2020-02-01']

fig_px_scatter = px.scatter(df_change_0_1, x="Open__change", y="Open__change_right", labels={
                        "Open__change": list_symbols[0],
                        "Open__change_right": list_symbols[1],
                    }).update_traces(mode="markers")
fig_go_Figure = go.Figure(data=fig_px_scatter)
fig_go_Figure.update_layout(title="Relative Change (%)")
fig_go_Figure.update_layout(title_x=0.5)
l1 = df_change_0_1["Open__change"].to_list()
l2 = df_change_0_1["Open__change_right"].to_list()
xy_max = max(max(l1), max(l2))*1.02
fig_go_Figure.update_layout(xaxis_range=[-1*xy_max, xy_max])
fig_go_Figure.update_layout(yaxis_range=[-1*xy_max, xy_max])

fig_dcc_Graph = dcc.Graph(id='id-f1', figure=fig_go_Figure)

fig_go_Figure.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig_go_Figure.show()


0.0386682

In [6]:
import numpy as np
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.figure_factory as ff

#%qtconsole

z = np.random.randint(0,6, size=(10, 10))
z_text = np.full(z.shape, '', dtype=str)

d = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f'}
class_mat = np.vectorize(d.get)(z)

fig = ff.create_annotated_heatmap(z, annotation_text=z_text,
                                  text=class_mat, hoverinfo='text', colorscale='Viridis',
#                                   x = list('ABCDEFGHIJ'),
#                                   y = list('ABCDEFGHIJ')
                                 )
fig.layout.title = 'Semantic Segmentation'

# My suggestions:
fig.data[0]['hoverinfo'] = 'all'
#fig['layout']['yaxis']['scaleanchor']='x'

fig['layout']['xaxis']['gridcolor'] = 'rgba(0, 0, 0, 0)'
fig['layout']['yaxis']['gridcolor'] = 'rgba(0, 0, 0, 0)'
fig['layout']['yaxis']['color'] = 'rgba(0, 0, 0, 0)'

fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')

fig.show()